In [ ]:
from utz import *

import plotly.express as px
import plotly.graph_objects as go

import sys
sys.executable

## Mapbox configs

In [ ]:
token = open(".mapbox-token").read()

osm = dict(mapbox=dict(style="open-street-map"))
usgs = dict(mapbox=dict(
    style="white-bg",
    layers=[{
        "below": 'traces',
        "sourcetype": "raster",
        "sourceattribution": "United States Geological Survey",
        "source": ["https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"]
    }],
))
stadia = dict(
    mapbox=dict(
        style="dark",
        accesstoken=token,
        layers=[{
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": '© <a href="https://stadiamaps.com/">Stadia Maps</a>, © <a href="https://openmaptiles.org/">OpenMapTiles</a> &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors',
            "source": ['https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png'],
        }],
    )
)
mapbox = dict(mapbox=dict(
    style="dark",
    accesstoken=token,
))

## NJDOT 2021 Crash SRIs/MPs

In [ ]:
%%time
from njdot.crashes import load, Crashes
c21 = Crashes.load(years=2021)
c21

In [ ]:
(~c21.lat.isna()).sum()

In [ ]:
c21.severity.value_counts()

In [ ]:
%%time
c21ll = c21.lls
c21ll

In [ ]:
%%time
nj = c21ll.nj
nnj = c21ll.nnj
len(nj), len(nnj)

In [ ]:
nnj

In [ ]:
severity_map = {
    "Property Damage": "yellow",
    "Injury": "orange",
    "Fatal": "red",
}

In [ ]:
from nj_crashes.geo import get_nj_points
from nj_crashes.utils import show
nj_points = get_nj_points()

In [ ]:
fig = px.scatter_mapbox(
    nnj.df,
    lat="lat", lon="lon",
    color='severity',
    hover_data=["County Name", "Date", "severity", "Crash Location", "sri", "mp"],
    color_discrete_map=severity_map,
    center=dict(lat=40.15, lon=-74.715),
    zoom=6.8,
    height=1000,
)
fig.add_trace(go.Scattermapbox(
    lat=nj_points.lat, lon=nj_points.lon,
    name='NJ',
    showlegend=False,
    #color=nj_points.name,
    #hover_name="City",
    #hover_data=["name"],
))
legend_bgcolor = '50'
fig.update_layout(
    **mapbox,
    title=dict(
        text="Erroneous crash lat/lons",
        x=0.5, y=0.98,
        xanchor='center', yanchor='top',
        font=dict(size=32, color="white")
    ),
    legend=dict(
        title=dict(text=''),
        x=0.98, y=0.98,
        xanchor="right", yanchor="top",
        font=dict(
            size=14,
            color="white"
        ),
        bgcolor=f"rgba({legend_bgcolor},{legend_bgcolor},{legend_bgcolor},0.8)",
        bordercolor="white",
        borderwidth=2,
    ),
    margin={"r":0,"t":0,"l":0,"b":0},
)
show(fig, w=800, h=800)

In [ ]:
c21ll[c21ll.lon > 0]

In [ ]:
c21ll.lon.dropna().astype(int).value_counts().sort_index()

### Plot crashes

In [ ]:
nj.severity.value_counts()

In [ ]:
fig = px.scatter_mapbox(
    nj.df,
    lat="lat", lon="lon",
    color='severity',
    #hover_name="City",
    hover_data=["County Name", "Date", "sri", "mp"],
    color_discrete_map=severity_map,
    center=dict(lat=40.29, lon=-74.715),
    zoom=7.4,
    height=1000,
)
legend_bgcolor = '50'
fig.update_layout(
    **mapbox,
    title=dict(
        text="NJ Crashes (2021)",
        x=0.5, y=0.95,
        xanchor='center', yanchor='top',
        font=dict(size=32, color="white")
    ),
    legend=dict(
        x=0.98, y=0.90,
        xanchor="right", yanchor="top",
        font=dict(
            size=14,
            color="white"
        ),
        bgcolor=f"rgba({legend_bgcolor},{legend_bgcolor},{legend_bgcolor},0.8)",
        bordercolor="white",
        borderwidth=2,
    ),
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_image('nj-crashes-mapbox.png', width=800, height=1000)
fig.write_json('nj-crashes-mapbox.json')
show(fig, w=800, h=800)

In [ ]:
c21ll[['lat', 'lon']].value_counts().value_counts().sort_index()

In [ ]:
nj.to_parquet('nj-2021-if.parquet')

In [ ]:
nj